[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](http://colab.research.google.com/github/angelosalton/data-notebooks/blob/master/web-scraping-pyspark-analysis.ipynb)

# A tarefa

Esse código tem como objetivo obter dados sobre trabalhos acadêmicos no [Locus UFV](https://www.locus.ufv.br/), o repositório de dissertações e teses da Universidade Federal de Viçosa, usando [Selenium](https://www.selenium.dev/about/), uma biblioteca para *web scraping*, ou raspagem de dados. Eu uso o Chrome como o navegador a ser automatizado, nesse caso é preciso configurar o `chromedriver` adequadamente.

# Configuração

In [ ]:
import sys

!pip install selenium
#!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver',chrome_options=chrome_options)

# Scraping

In [ ]:
driver.get('https://www.locus.ufv.br/handle/123456789/2/recent-submissions')

O código a seguir obtém o título, autor e uma pequena parte do abstract/resumo. A partir daí, é possível fazer análises de texto e *clustering*.

In [ ]:
# store entries
entries = []

err = False
while err == False:
    # list of entries per page
    elems = driver.find_element_by_css_selector('.ds-artifact-list')

    # there are 20 entries per page
    for i in range(20):
        entries.append(elems.find_elements_by_css_selector('li.ds-artifact-item:nth-child({0})'.format(i+1))[0].text)
    
    # navigate to next page until there are no more pages
    try:
        nextpage = driver.find_element_by_css_selector('div.pagination:nth-child(1) > ul:nth-child(2) > li:nth-child(2) > a:nth-child(1)')
        nextpage.click()
    except NoSuchElementException:
        err = True
        print('Job complete.')

Finalmente, salvando o resultado em um `DataFrame`:

In [ ]:
import pandas as pd

df = pd.DataFrame(list(map(lambda x: x.split('\n'), entries)), columns=['title','author','abstract'])
df.head()